# This is to prepare the both_sexes IPDGC summarystats
[Reference this](https://github.com/neurogenetics/Autosomal-sex-differences-PDv2/blob/main/Meta%20analyses%20of%20Results.md)

METAL output was joined with HRC hg19 no_multi_allelic REF ALT. 

Filter: MixFreq>0.01, MaxFreq<0.99 HetDF > 2/3 * max(HetDF)

In [86]:
library(data.table)
library(tidyverse)

In [31]:
d = fread('/data/CARD/PD/summary_stats/META5_no23noUKBB.tbl.gz') %>%
    mutate(MarkerName=sub('chr', '', MarkerName))
RS=fread('/data/CARD/GENERAL/HRC_RS_conversion_final.txt') %>%
    rename(MarkerName=POS)
dt = left_join(d, RS, by='MarkerName')
dim(d)
dim(RS)
dim(dt)

dtf = dt %>% 
    filter(MinFreq>0.01, MaxFreq<0.99, HetDf>6) %>% # HetDf = max 11 * 2/3 floor
    arrange(MarkerName)

dim(dtf)

fwrite(dtf, 
       '/data/CARD/projects/sexMRforPD/common/BOTH_SEXES_PD_filtered_sumstats_NO_UKB_AT_ALL_no_multi_allelics.txt', 
       sep='\t')

[1] 17409785       15

[1] 33619058        4

[1] 17439466       18

[1] 6706231      18

In [87]:
# create files for analysis 
d = dtf %>% 
    mutate(FreqDiff = MaxFreq - MinFreq) %>%
    filter(!is.na(ID), FreqDiff<0.15, HetISq<80, Effect<5, StdErr<5)
dim(d)

dt = d %>% 
    mutate(flip = case_when(
        (REF==toupper(Allele2))&(ALT==toupper(Allele1)) ~ 0,
        (REF==toupper(Allele1))&(ALT==toupper(Allele2)) ~ 1
    )) %>%
    mutate(
        SNP=ID,
        A1=ALT,
        A2=REF,
        freq=case_when(flip==0~Freq1, flip==1~1-Freq1),
        b=case_when(flip==0~Effect, flip==1~-Effect),
        se=StdErr,
        p=`P-value`,
        N = 43826
    ) %>%
    filter(!is.na(flip)) # remove unbigous variants

dim(dt)
table(dt$flip)

dt %>% filter(`P-value`<5e-8) %>% select(ID) %>%
    fwrite(., '/data/CARD/projects/sexMRforPD/common/both_sexes_pd_iv_5e8.txt', sep='\t', col.names = F)
dt %>% filter(`P-value`<1e-4) %>% select(ID) %>%
    fwrite(., '/data/CARD/projects/sexMRforPD/common/both_sexes_pd_iv_1e4.txt', sep='\t', col.names = F)
fwrite(dt[,c('SNP', 'A1', 'A2', 'freq', 'b', 'se', 'p', 'N')],
       '/data/CARD/projects/sexMRforPD/common/both_sexes_pd_gsmr.raw', sep='\t')

dt2 = dt %>% separate(MarkerName, sep=':', into=c('chr', 'BP')) %>%
    rename(beta=b, pvalue=p)
fwrite(dt2[,c('SNP', 'chr', 'BP', 'A1', 'A2', 'beta', 'se', 'pvalue')],
       '/data/CARD/projects/sexMRforPD/common/both_sexes_pd_mrldp.raw', sep='\t')

[1] 6586565      19

[1] 6582857      28


      0       1 
3558782 3024075 